In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing

In [151]:
# Import Dataset
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Age Prediction

In [152]:
# 미스터, 미세스 만 빼버리기
train['Name'] = train['Name'].str.split(',').str[1].str.split('.').str[0]
le3 = preprocessing.LabelEncoder()
le3.fit(train['Name'])
train['Name'] = le3.transform(train['Name'])


#결측치 처리 (Embarked)
train['Embarked'].fillna('C',inplace=True)

#Cabin 알파벳으로 변환
train['Cabin_alpha'] = train[train['Cabin'].isnull()==False]['Cabin'].map(lambda x: x[0])
train['Cabin_alpha'].fillna("N",inplace=True)

#Group 변수 생성
#Family size랑 dup_count 중 큰걸로 fare 나누기.
train['Family_size'] = train['SibSp']+train['Parch']+1
train['dup_count'] = train.groupby(['Ticket'])['Ticket'].transform('count')
train['Group'] = ~((train['Family_size'] == 1) & (train['dup_count'] == 1))
train['Group_count'] = train[['Family_size','dup_count']].max(axis=1)
train['Fare_ind'] = train['Fare']/train['Group_count']



# Categorical Feature Encoding
le = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le.fit(train['Embarked'])
le2.fit(train['Cabin_alpha'])
train['Embarked']=le.transform(train['Embarked'])
train['Cabin_alpha']=le2.transform(train['Cabin_alpha'])
train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'male' else 0)


df_train = train[~train['Age'].isnull()]
# Age Binning
df_train['Age'] = df_train['Age'].apply(lambda x: 0 if x < 17 else(0 if x >49 else 1))

/Users/danielhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [96]:
df_train.corr().Age

PassengerId    0.026097
Survived      -0.075247
Pclass         0.051349
Name           0.334746
Sex            0.050095
Age            1.000000
SibSp         -0.308304
Parch         -0.229330
Fare          -0.034116
Embarked       0.061914
Cabin_alpha    0.059787
Family_size   -0.325074
dup_count     -0.286409
Group         -0.201068
Group_count   -0.294027
Fare_ind      -0.004921
Name: Age, dtype: float64

In [99]:
df_train[['Name','Family_size','dup_count']].corr()

,Name,Family_size,dup_count
Name,1.000000,-0.191291,-0.167073
Family_size,-0.191291,1.000000,0.783988
dup_count,-0.167073,0.783988,1.000000


In [153]:
train_X = df_train[['Name','SibSp','Parch']]
train_y = df_train.Age

In [137]:
xg = xgb.XGBClassifier(random_seed=0,n_estimators=200)
parameters = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.0001, 0.001, 0.01, 0.1]
        }
clf = RandomizedSearchCV(xg, parameters,n_iter=50, cv=5)
clf.fit(train_X, train_y)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=200,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_seed=0, random_state=0,
                                           reg_alpha=0, reg_...
                                           silent=True, subsample=1),
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                   

In [138]:
#with all features model
print(clf.best_params_)
print(clf.best_score_)

{'subsample': 1.0, 'min_child_weight': 10, 'max_depth': 5, 'learning_rate': 0.0001, 'gamma': 2, 'colsample_bytree': 1.0}
0.834733893557423


In [104]:
#with age corr >0.3 features model
print(clf.best_params_)
print(clf.best_score_)

{'subsample': 1.0, 'min_child_weight': 10, 'max_depth': 5, 'learning_rate': 0.001, 'gamma': 2, 'colsample_bytree': 1.0}
0.834733893557423


In [139]:


df_test = train[train['Age'].isnull()]
test_X = df_test[['Name','SibSp','Parch']]
y = clf.predict(test_X)

In [154]:
train.loc[~train['Age'].isnull(),'Age'] = train[~train['Age'].isnull()]['Age'].apply(lambda x: 0 if x < 17 else(0 if x >49 else 1))
train.loc[train['Age'].isnull(),'Age'] = y
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_alpha,Family_size,dup_count,Group,Group_count,Fare_ind
0,1,0,3,11,1,1.0,1,0,A/5 21171,7.2500,NaN,2,7,2,1,True,2,3.625000
1,2,1,1,12,0,1.0,1,0,PC 17599,71.2833,C85,0,2,2,1,True,2,35.641650
2,3,1,3,8,0,1.0,0,0,STON/O2. 3101282,7.9250,NaN,2,7,1,1,False,1,7.925000
3,4,1,1,12,0,1.0,1,0,113803,53.1000,C123,2,2,2,2,True,2,26.550000
4,5,0,3,11,1,1.0,0,0,373450,8.0500,NaN,2,7,1,1,False,1,8.050000
5,6,0,3,11,1,1.0,0,0,330877,8.4583,NaN,1,7,1,1,False,1,8.458300
6,7,0,1,11,1,0.0,0,0,17463,51.8625,E46,2,4,1,1,False,1,51.862500
7,8,0,3,7,1,0.0,3,1,349909,21.0750,NaN,2,7,5,4,True,5,4.215000
8,9,1,3,12,0,1.0,0,2,347742,11.1333,NaN,2,7,3,3,True,3,3.711100
9,10,1,2,12,0,0.0,1,0,237736,30.0708,NaN,0,7,2,2,True,2,15.035400


In [155]:
train.Age.value_counts()

1.0    703
0.0    188
Name: Age, dtype: int64

# Modeling

In [156]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_alpha,Family_size,dup_count,Group,Group_count,Fare_ind
0,1,0,3,11,1,1.0,1,0,A/5 21171,7.2500,NaN,2,7,2,1,True,2,3.625000
1,2,1,1,12,0,1.0,1,0,PC 17599,71.2833,C85,0,2,2,1,True,2,35.641650
2,3,1,3,8,0,1.0,0,0,STON/O2. 3101282,7.9250,NaN,2,7,1,1,False,1,7.925000
3,4,1,1,12,0,1.0,1,0,113803,53.1000,C123,2,2,2,2,True,2,26.550000
4,5,0,3,11,1,1.0,0,0,373450,8.0500,NaN,2,7,1,1,False,1,8.050000
5,6,0,3,11,1,1.0,0,0,330877,8.4583,NaN,1,7,1,1,False,1,8.458300
6,7,0,1,11,1,0.0,0,0,17463,51.8625,E46,2,4,1,1,False,1,51.862500
7,8,0,3,7,1,0.0,3,1,349909,21.0750,NaN,2,7,5,4,True,5,4.215000
8,9,1,3,12,0,1.0,0,2,347742,11.1333,NaN,2,7,3,3,True,3,3.711100
9,10,1,2,12,0,0.0,1,0,237736,30.0708,NaN,0,7,2,2,True,2,15.035400


In [157]:
#모델에 사용하지 않을 변수 제거
train_X_surv = train.drop(['PassengerId','Survived','Cabin','Ticket','Fare','Family_size','dup_count','SibSp','Parch'],axis=1)
train_y_surv = train.Survived

In [158]:
train_X_surv

,Pclass,Name,Sex,Age,Embarked,Cabin_alpha,Group,Group_count,Fare_ind
0,3,11,1,1.0,2,7,True,2,3.625000
1,1,12,0,1.0,0,2,True,2,35.641650
2,3,8,0,1.0,2,7,False,1,7.925000
3,1,12,0,1.0,2,2,True,2,26.550000
4,3,11,1,1.0,2,7,False,1,8.050000
5,3,11,1,1.0,1,7,False,1,8.458300
6,1,11,1,0.0,2,4,False,1,51.862500
7,3,7,1,0.0,2,7,True,5,4.215000
8,3,12,0,1.0,2,7,True,3,3.711100
9,2,12,0,0.0,0,7,True,2,15.035400


In [159]:
xg = xgb.XGBClassifier(random_seed=0,n_estimators=400)
parameters = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.0001, 0.001, 0.01, 0.1]
        }
clf = RandomizedSearchCV(xg, parameters,n_iter=100, cv=5,random_state=0)
clf.fit(train_X_surv, train_y_surv)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=400,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_seed=0, random_state=0,
                                           reg_alpha=0, reg_...
                                           silent=True, subsample=1),
                   iid='warn', n_iter=100, n_jobs=None,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                  

In [160]:
clf.best_params_

{'subsample': 0.8,
 'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 1,
 'colsample_bytree': 0.8}

In [161]:
clf.best_score_

0.8383838383838383

In [162]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_alpha,Family_size,dup_count,Group,Group_count,Fare_ind
0,1,0,3,11,1,1,1,0,A/5 21171,7.2500,NaN,2,7,2,1,True,2,3.625000
1,2,1,1,12,0,1,1,0,PC 17599,71.2833,C85,0,2,2,1,True,2,35.641650
2,3,1,3,8,0,1,0,0,STON/O2. 3101282,7.9250,NaN,2,7,1,1,False,1,7.925000
3,4,1,1,12,0,1,1,0,113803,53.1000,C123,2,2,2,2,True,2,26.550000
4,5,0,3,11,1,1,0,0,373450,8.0500,NaN,2,7,1,1,False,1,8.050000
6,7,0,1,11,1,0,0,0,17463,51.8625,E46,2,4,1,1,False,1,51.862500
7,8,0,3,7,1,0,3,1,349909,21.0750,NaN,2,7,5,4,True,5,4.215000
8,9,1,3,12,0,1,0,2,347742,11.1333,NaN,2,7,3,3,True,3,3.711100
9,10,1,2,12,0,0,1,0,237736,30.0708,NaN,0,7,2,2,True,2,15.035400
10,11,1,3,8,0,0,1,1,PP 9549,16.7000,G6,2,6,3,2,True,3,5.566667


# 생성변수 노약자

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_alpha,Family_size,dup_count,Group,Group_count,Fare_ind
0,1,0,3,11,1,1.0,1,0,A/5 21171,7.2500,NaN,2,7,2,1,True,2,3.625000
1,2,1,1,12,0,1.0,1,0,PC 17599,71.2833,C85,0,2,2,1,True,2,35.641650
2,3,1,3,8,0,1.0,0,0,STON/O2. 3101282,7.9250,NaN,2,7,1,1,False,1,7.925000
3,4,1,1,12,0,1.0,1,0,113803,53.1000,C123,2,2,2,2,True,2,26.550000
4,5,0,3,11,1,1.0,0,0,373450,8.0500,NaN,2,7,1,1,False,1,8.050000
5,6,0,3,11,1,1.0,0,0,330877,8.4583,NaN,1,7,1,1,False,1,8.458300
6,7,0,1,11,1,0.0,0,0,17463,51.8625,E46,2,4,1,1,False,1,51.862500
7,8,0,3,7,1,0.0,3,1,349909,21.0750,NaN,2,7,5,4,True,5,4.215000
8,9,1,3,12,0,1.0,0,2,347742,11.1333,NaN,2,7,3,3,True,3,3.711100
9,10,1,2,12,0,0.0,1,0,237736,30.0708,NaN,0,7,2,2,True,2,15.035400


In [216]:
corr_df = pd.read_csv('train.csv')
corr_df = corr_df[['Age','Sex','Survived']]

In [217]:
#corr_df['Age'].fillna(corr_df['Age'].mean(),inplace=True)
corr_df=corr_df[~corr_df['Age'].isnull()]

In [218]:
corr = []
rang = []
for low in range(5,20):
    rang.append(str(low)+'< Age' )
    corr.append(corr_df.apply(lambda x: 0 if x['Age'] < low or x['Sex'] == 'female' else 1, axis=1).corr(corr_df.Survived))

In [219]:
pd.DataFrame({'Loop_info':rang,'Correlation':corr}).sort_values(['Correlation'])

,Loop_info,Correlation
2,7< Age,-0.566274
0,5< Age,-0.563150
1,6< Age,-0.563150
5,10< Age,-0.562326
4,9< Age,-0.561352
3,8< Age,-0.560893
8,13< Age,-0.560751
9,14< Age,-0.560751
6,11< Age,-0.559683
7,12< Age,-0.557585


0      1
1      0
2      0
3      0
4      1
5      1
6      0
7      0
8      0
9      0
10     0
11     0
12     1
13     1
14     0
15     0
16     0
17     1
18     0
19     0
20     1
21     1
22     0
23     1
24     0
25     0
26     1
27     1
28     0
29     1
      ..
861    1
862    0
863    0
864    1
865    0
866    0
867    1
868    1
869    0
870    1
871    0
872    1
873    1
874    0
875    0
876    1
877    1
878    1
879    0
880    0
881    1
882    0
883    1
884    1
885    0
886    1
887    0
888    0
889    1
890    1
Length: 891, dtype: int64